# Flood Estimate Code

This code produces flood heights for given storm parameters. The code calculates surface volume functions that can relate the volume of water to the flood height at a given division. Using Mannning's equation, it takes topographic data and storm parameter data to find the velocity and subsequently the volume of the water. This volumes are then redistributed by propagating to the nearby divisions, and the final flood heights for each division are determined.


Inputs: surface data (topography, roughness, slope, surface volume, and surface volume grouped), flood data for cold and warm storms (peak, surge, time), and wall data for cold and warm storms.

Outputs: Flood heights for cold and warm storms with and without flood wall protection in csv format.

## Imports and Functions

In [ ]:
import numpy as np
import pandas as pd
import time
import csv
import glob                                                                                                
import datetime
from scipy.optimize import curve_fit

In [ ]:
import json, yaml, os
cfg = yaml.safe_load(open("config.yml"))
num_divs = cfg["num_divs"]
DEM_DIR = cfg["dem_folder"]
SV_OUT  = cfg["surface_volume_out"]
PROJECT = cfg["project_name"]

In [10]:
# essentials
ftm = 0.3048 # feet to meter conversion factor
nt  = 24  # split number of time history of storm surge

In [ ]:
def fit_piecewise_alpha_beta(surfaceV, H):
    # surfaceV, H are 1D arrays ordered by increasing V
    # sliding windows of width 2, step 1
    blocks = []
    for w in range(1, len(surfaceV)):
        x = surfaceV[w-1:w+1]
        y = H[w-1:w+1]
        # linearize: y = a*sqrt(x) + b*x  -> fit via least squares on phi=[sqrt(x), x]
        Phi = np.vstack([np.sqrt(x), x]).T
        a,b = np.linalg.lstsq(Phi, y, rcond=None)[0]
        blocks.append({"Vmax": surfaceV[w], "coef": (a,b)})
    return blocks

def eval_piecewise(blocks, V):
    # choose the first block where V <= Vmax, else use last
    for blk in blocks:
        if V <= blk["Vmax"]:
            a,b = blk["coef"]
            return a*np.sqrt(V) + b*V
    a,b = blocks[-1]["coef"]

In [ ]:
## Functions

# curve fitting

# --- model stays the same ---
def func_fit(surfaceV, a, b):
    return a*np.sqrt(surfaceV) + b*surfaceV

class SVPieces:
    """
    Holds piecewise fits for the Surface-Volume curve.
    pieces: list of dicts with {'Vmax': float, 'coef': (a,b)}
    """
    def __init__(self, pieces):
        self.pieces = pieces

    def eval(self, V):
        """Evaluate h(V) using the first piece with V <= Vmax (else last)."""
        V = np.asarray(V)
        if V.ndim == 0:
            return self._eval_scalar(float(V))
        out = np.empty_like(V, dtype=float)
        for idx, v in np.ndenumerate(V):
            out[idx] = self._eval_scalar(float(v))
        return out

    def _eval_scalar(self, V):
        for blk in self.pieces:
            if V <= blk['Vmax']:
                a, b = blk['coef']
                return max(0.0, a*np.sqrt(V) + b*V)  # clamp to 0 like your code
        a, b = self.pieces[-1]['coef']
        return max(0.0, a*np.sqrt(V) + b*V)

def SurfaceVolFunc(surfaceV, H, window=2, step=1):
    """
    Build piecewise fits along the (surfaceV, H) curve.

    Parameters
    ----------
    surfaceV : 1D array (ascending volumes)
    H        : 1D array (heights)
    window   : int, window size in points (2 replicates your original)
    step     : int, slide step

    Returns
    -------
    SVPieces object with .pieces = [{'Vmax': ..., 'coef': (a,b)}, ...]
    """
    surfaceV = np.asarray(surfaceV, dtype=float)
    H        = np.asarray(H, dtype=float)

    # ensure strictly ascending V for curve_fit stability
    order = np.argsort(surfaceV)
    surfaceV, H = surfaceV[order], H[order]

    pieces = []
    n = len(surfaceV)
    end = n - window + 1
    if end < 1:
        raise ValueError("Not enough points for the chosen window size.")

    for start in range(0, end, step):
        stop = start + window
        x = surfaceV[start:stop]
        y = H[start:stop]
        # avoid identical x causing singular Jacobian
        if np.allclose(x.max(), x.min()):
            continue
        (a, b), _ = curve_fit(func_fit, x, y, maxfev=10000)
        pieces.append({'Vmax': x[-1], 'coef': (float(a), float(b))})

    # ensure last piece covers the tail
    if pieces and pieces[-1]['Vmax'] < surfaceV[-1]:
        x = surfaceV[-window:]
        y = H[-window:]
        (a, b), _ = curve_fit(func_fit, x, y, maxfev=10000)
        pieces.append({'Vmax': surfaceV[-1], 'coef': (float(a), float(b))})

    return SVPieces(pieces)

def FloodHeight(surfaceV, slope, roughness, SVpieces,
                time1_w, time2_w, cpi1_w, cpi2_w, nt, elev, fid, l, ssh, i):
    """
    Calculates flood heights for a given division without redistribution.
    SVpieces: SVPieces object from SurfaceVolFunc(surfaceV, H, ...)
    """
    sec1_w = time1_w * 60**2
    sec2_w = time2_w * 60**2
    h1_w   = cpi1_w
    h2_w   = cpi2_w
    V_new  = 0.0
    v_new  = np.zeros(nt*2)

    # incoming volume for this division
    for k in fid:
        ds1 = np.tile(sec1_w[1] - sec1_w[0], (nt, 1)).T
        ds2 = np.tile(sec2_w[1] - sec2_w[0], (nt, 1)).T
        ds  = np.concatenate((ds1, ds2), axis=1)

        h1_new = h1_w - elev[k]
        h2_new = h2_w - elev[k]
        h_new  = np.concatenate((h1_new, h2_new), axis=0)

        pos = h_new > 0
        v_new[pos]  = ((l * h_new[pos]) / (l + (2*h_new[pos])))**(2./3.) * (slope**0.5) / roughness
        v_new[~pos] = 0.0

        V_new += np.sum(h_new * l * v_new * ds)

    # evaluate height from the piecewise curve
    if V_new <= 0:
        return 0.0, V_new

    fld_h = float(SVpieces.eval(V_new))
    # fld_h = min(fld_h, float(ssh))
    return fld_h, V_new

def FloodHeightWall(surfaceV, ParWall, slope, roughness, SVpieces,
                    time1_w, time2_w, cpi1_w, cpi2_w, nt, elev, fid, l, ssh, i):
    sec1_w = time1_w * 60**2
    sec2_w = time2_w * 60**2
    h1_w   = cpi1_w
    h2_w   = cpi2_w
    V_new  = 0.0
    v_new  = np.zeros(nt*2)

    for k in fid:
        ds1 = np.tile(sec1_w[1] - sec1_w[0], (nt, 1)).T
        ds2 = np.tile(sec2_w[1] - sec2_w[0], (nt, 1)).T
        ds  = np.concatenate((ds1, ds2), axis=1)

        h1_new = h1_w - elev[k]
        h2_new = h2_w - elev[k]
        h_new  = np.concatenate((h1_new, h2_new), axis=0)

        pos = h_new > 0
        if k in ParWall:
            cwr = 0.611 + 0.075 * (h_new[pos] / elev[k])
            v_new[pos] = ((l * h_new[pos]) / (l + (2*h_new[pos])))**(2./3.) * (slope**0.5) / roughness * cwr
        else:
            v_new[pos] = ((l * h_new[pos]) / (l + (2*h_new[pos])))**(2./3.) * (slope**0.5) / roughness
        v_new[~pos] = 0.0

        V_new += np.sum(h_new * l * v_new * ds)

    if V_new <= 0:
        return 0.0, V_new

    fld_h = float(SVpieces.eval(V_new))
    # fld_h = min(fld_h, float(ssh))  # if the same surge cap here
    return fld_h, V_new

def FloodTravel(ssh, V_new, SVpieces):
    """
    Given redistributed volume for one division, compute new flood height.
    """
    if V_new <= 0:
        return 0.0
    h = float(SVpieces.eval(V_new))
    return min(h, float(ssh))

def FloodTravelSectGroup(ssh, groups,  # dict: j -> list of division indices to aggregate with j (including j if desired)
                         V,            # per-division volumes from the first pass
                         SVpieces_by_div):
    """
    Propagate flooding via user-defined groups.
    Returns:
      fld_h_sect: per-division redistributed heights
      V_sect_avr: per-division averaged volume used to compute height
    """
    n = len(V)
    fld_h_sect = np.zeros(n)
    V_sect_avr = np.zeros(n)

    for j in range(n):
        members = groups.get(j, [j])          # default to itself
        V_sum   = float(np.sum(V[members]))
        # "average per member" semantics like before:
        V_avg   = V_sum / max(1, len(members))
        V_sect_avr[j] = V_avg

        # Use SV curve for division j (mirrors original logic using SVfg* per i)
        fld_h_sect[j] = FloodTravel(ssh, V_sum, SVpieces_by_div[j])

    fld_h_sect[fld_h_sect < 0] = 0.0
    return fld_h_sect, V_sect_avr


## Import Data

In [ ]:
# --- Config + imports ---
import os, re, glob, json, yaml
import numpy as np
import pandas as pd

# Load project config (edit to your path/name if needed)
cfg = yaml.safe_load(open("config.yml", "r"))

# Required keys (examples):
# cfg = {
#   "project_name": "MyCity_div30",
#   "elevation_csv": "Data/MyCity_div_low.csv",
#   "roughness_csv": "Data/MyCity_Roughness.csv",
#   "slope_csv": "Data/MyCity_Slope.csv",
#   "sv_individual_dir": "Data/NewSurfaceVolumeCombined",
#   "sv_grouped_dir": "Data/NewSurfaceVolumeGrouped",
#   "sv_individual_glob": "*.csv",
#   "sv_grouped_glob": "*.csv",
#   "adjacency": {"type":"groups", "groups_file":"Data/Groups.json"},
#   "coast_segment_length_m": 100
# }

# --- Load base topography / attributes ---
topo = pd.read_csv(cfg["elevation_csv"])

# Try to standardize column names:
# Expect elevation column like "MEAN2" (fallbacks allowed)
elev_col = next((c for c in topo.columns if c.lower() in {"mean2","elev","elevation"}), None)
fid_col  = next((c for c in topo.columns if c.lower() in {"fid","id","cellid","index"}), None)
div_col  = next((c for c in topo.columns if c.lower().startswith("div")), None)
assert elev_col and fid_col and div_col, "Can't find elevation/FID/DIV columns in your topography CSV."

elev   = topo[elev_col].to_numpy()
fid    = topo[fid_col].to_numpy()
div_id = topo[div_col].to_numpy()
num_divs = int(np.unique(div_id).size)

# Roughness & slope (vector or scalar both supported)
roughness = pd.read_csv(cfg["roughness_csv"]).squeeze("columns").to_numpy()
slope     = pd.read_csv(cfg["slope_csv"]).squeeze("columns").to_numpy()

# Coastal segment length (can be scalar or per-division; here scalar from config)
l = float(cfg.get("coast_segment_length_m", 100.0))

# Discretized heights used to build surface-volume curves (keep your current sampling)
H = np.append(np.linspace(0, 3, 13), np.linspace(3.5, 7, 8))

# --- Helper: natural sort by any number in filename, and extract division index if present ---
num_re = re.compile(r"(\d+)")
def naturalsort_key(path):
    # Split into chunks of digits/non-digits for human-like sorting
    return [int(t) if t.isdigit() else t.lower() for t in re.split(r'(\d+)', os.path.basename(path))]

def extract_div_index(path):
    # Try to find a division index in the filename; fallback to None
    m = num_re.findall(os.path.basename(path))
    return int(m[-1]) if m else None

# --- Find individual & grouped SV files (no hard-coded LMN_div18 prefixes) ---
indiv_files  = sorted(
    glob.glob(os.path.join(cfg["sv_individual_dir"], cfg.get("sv_individual_glob","*.csv"))),
    key=naturalsort_key
)
group_files  = sorted(
    glob.glob(os.path.join(cfg["sv_grouped_dir"], cfg.get("sv_grouped_glob","*.csv"))),
    key=naturalsort_key
)

# Sanity checks (optional)
if len(indiv_files) and len(indiv_files) != num_divs:
    print(f"Warning: found {len(indiv_files)} individual SV files but {num_divs} divisions.")
if len(group_files) and len(group_files) != num_divs:
    print(f"Note: found {len(group_files)} grouped SV files (optional), divisions={num_divs}.")

# --- Build SV piecewise models (individual) ---
SVpieces_by_div = [None] * num_divs
SV_all = []  # optional matrix of volumes per div (for diagnostics)

for path in indiv_files:
    # Read volume column (keep name flexible)
    df = pd.read_csv(path)
    vol_col = next((c for c in df.columns if c.lower() in {"volume","vol","v"}), None)
    assert vol_col, f"No 'volume' column in {path}"
    surfaceV = df[vol_col].to_numpy()

    # Decide which division index this file belongs to
    j = extract_div_index(path)
    if j is None or j >= num_divs:
        # fallback: assign next empty slot
        j = next(idx for idx,v in enumerate(SVpieces_by_div) if v is None)

    # Build piecewise fit (window=2, step=1 replicates legacy 20-slice behavior)
    SVpieces_by_div[j] = SurfaceVolFunc(surfaceV, H, window=2, step=1)
    SV_all.append(surfaceV)

# Stack volumes per division if desired (rows aligned to 0..num_divs-1)
if SV_all:
    # ensure ordering matches division index 0..num_divs-1
    reorder = [i for i,v in enumerate(SVpieces_by_div) if v is None]
    assert not reorder, "Some divisions are missing individual SV files."
    SV_all = np.vstack(SV_all)  # shape (num_divs, len(H))

# --- Build SV piecewise models (grouped) if provided ---
SVpieces_grouped_by_div = [None] * num_divs
if group_files:
    for path in group_files:
        df = pd.read_csv(path)
        vol_col = next((c for c in df.columns if c.lower() in {"volume","vol","v"}), None)
        assert vol_col, f"No 'volume' column in {path}"
        surfaceVg = df[vol_col].to_numpy()

        j = extract_div_index(path)
        if j is None or j >= num_divs:
            j = next(idx for idx,v in enumerate(SVpieces_grouped_by_div) if v is None)

        SVpieces_grouped_by_div[j] = SurfaceVolFunc(surfaceVg, H, window=2, step=1)

# --- Load user-defined interdependencies / groups ---
# Replace all hard-coded sect0/sect1/... logic with a single groups mapping.
groups = {}  # dict[int, list[int]]

adj_cfg = cfg.get("adjacency", {})
if adj_cfg.get("type","groups") == "groups":
    with open(adj_cfg["groups_file"], "r") as f:
        raw = json.load(f)
    # normalize keys to int, values to list[int]
    for k,v in raw.items():
        j = int(k)
        groups[j] = [int(x) for x in v]
elif adj_cfg.get("type") == "graph":
    # If a graph is provided, you can derive groups from neighbors or radii as you prefer
    with open(adj_cfg["graph_file"], "r") as f:
        graph = json.load(f)  # e.g., {"0":[1,2], "1":[0,2], ...}
    for k, nbrs in graph.items():
        j = int(k)
        groups[j] = [j] + [int(x) for x in nbrs]   # example: self + 1-hop neighbors
else:
    # default: each division forms its own group
    groups = {j:[j] for j in range(num_divs)}


/tmp/ipykernel_78501/2785758856.py:21: OptimizeWarning: Covariance of the parameters could not be estimated
  popt1, pcov1 = curve_fit(func_fit, surfaceV[a1:a2], H[a1:a2])
/tmp/ipykernel_78501/2785758856.py:22: OptimizeWarning: Covariance of the parameters could not be estimated
  popt2, pcov2 = curve_fit(func_fit, surfaceV[b1:b2], H[b1:b2])
/tmp/ipykernel_78501/2785758856.py:23: OptimizeWarning: Covariance of the parameters could not be estimated
  popt3, pcov3 = curve_fit(func_fit, surfaceV[c1:c2], H[c1:c2])
/tmp/ipykernel_78501/2785758856.py:24: OptimizeWarning: Covariance of the parameters could not be estimated
  popt4, pcov4 = curve_fit(func_fit, surfaceV[d1:d2], H[d1:d2])
/tmp/ipykernel_78501/2785758856.py:25: OptimizeWarning: Covariance of the parameters could not be estimated
  popt5, pcov5 = curve_fit(func_fit, surfaceV[e1:e2], H[e1:e2])
/tmp/ipykernel_78501/2785758856.py:26: OptimizeWarning: Covariance of the parameters could not be estimated
  popt6, pcov6 = curve_fit(func_

## Flood Simulation - No Protective Measures

Flood simulation takes in storm peaks, surge heights, and time, as well as information about the terrain, and the surface volume functions from the Import Data cell to obtain the floodheights of individual cells and redistribute the flooding to adjacent cells. The resulting flood heights of the storm for cold and warm storms are outputted in a csv.  

In [ ]:
# --- Storm / run setup (division-agnostic) ---

import time
start_time = time.time()

storm_cfg = cfg.get("storm", {})
mode = storm_cfg.get("mode", "time_history")  # "time_history" or "peak_only"

if mode == "time_history":
    surge_csv   = storm_cfg["surge_csv"]                          # e.g., "Data/CO-OPS_8518750_met_hr.csv"
    surge_col   = storm_cfg.get("surge_column", "Verified (m)")   # override if your column differs
    surge       = pd.read_csv(surge_csv)[surge_col].to_numpy().astype(float)

    # split into two equal halves like the legacy code (e.g., 48 pts -> 24+24)
    nt = int(storm_cfg.get("nt", len(surge)//2))
    assert 2*nt <= len(surge), "Surge series shorter than expected (need 2*nt points)."

    time1_w = np.arange(0, nt, dtype=float)
    time2_w = np.arange(nt, 2*nt, dtype=float)
    cpi1_w  = surge[:nt]
    cpi2_w  = surge[nt:2*nt]
    peak_w  = float(np.max(surge))

elif mode == "peak_only":
    peak_w   = float(storm_cfg["peak_m"])          # required in this mode
    nt       = int(storm_cfg.get("nt", 24))
    # synthesize a simple half-sine hydrograph rising and falling over 2*nt samples
    t        = np.linspace(0, np.pi, 2*nt)
    surge    = peak_w * np.sin(t)
    time1_w  = np.arange(0, nt, dtype=float)
    time2_w  = np.arange(nt, 2*nt, dtype=float)
    cpi1_w   = surge[:nt]
    cpi2_w   = surge[nt:]
else:
    raise ValueError(f"Unknown storm mode: {mode}")

# optional SLR offsets in meters
slr = float(storm_cfg.get("slr_m", 0.0))
if slr:
    cpi1_w = cpi1_w + slr
    cpi2_w = cpi2_w + slr
    peak_w = peak_w + slr

# Preallocate outputs
fld_h_w = np.zeros(num_divs, dtype=float)  # heights (no-redistribution)
V_w     = np.zeros(num_divs, dtype=float)  # volumes per division

# --- First pass: per-division flood height/volume without redistribution ---

for j in range(num_divs):
    mask_j = (div_id == j)
    elev_j = elev[mask_j]
    fid_j  = fid[mask_j]

    # Pick the right SV curve set for this division (grouped optional)
    SVj = SVpieces_by_div[j]   # individual curve for division j

    # If your FloodHeight signature kept the extra 'surfaceV' arg, pass a dummy or the per-div series:
    surfaceV_dummy = SV_all[j] if 'SV_all' in locals() and len(SV_all) > j else np.array([])

    fld_h_w[j], V_w[j] = FloodHeight(
        surfaceV_dummy,           # not used by the refactored function; placeholder OK
        slope[j] if np.ndim(slope) else float(slope),
        roughness[j] if np.ndim(roughness) else float(roughness),
        SVj,
        time1_w, time2_w, cpi1_w, cpi2_w, nt,
        elev_j, fid_j,
        l, peak_w, j
    )

# --- Redistribution via user-defined groups ---

# Prefer grouped SV curves if you precomputed them; otherwise reuse individual
SV_for_redistribution = [
    (SVpieces_grouped_by_div[j] if SVpieces_grouped_by_div and SVpieces_grouped_by_div[j] is not None
     else SVpieces_by_div[j])
    for j in range(num_divs)
]

fld_h_w_sect_g, V_w_sect_avr = FloodTravelSectGroup(
    peak_w,                     # cap by local surge peak as in your original
    groups,                     # dict: j -> list of div indices to pool with j
    V_w,                        # volumes from first pass
    SV_for_redistribution       # per-division SV curve objects
)

# --- Save & report ---

out_csv = cfg.get("outputs", {}).get("single_storm_csv",
                                     "Sample_Output_Data/candidate/single_storm_flood_heights_w.csv")

os.makedirs(os.path.dirname(out_csv), exist_ok=True)
with open(out_csv, "w", newline="") as out:
    write = csv.writer(out)
    write.writerow(fld_h_w_sect_g)

elapsed_time = time.time() - start_time
print("Elapsed Time:", elapsed_time, "sec")
print("Complete!")
print(fld_h_w_sect_g)

Elapsed Time:  0.09606695175170898 sec
Complete!
[3.48 3.48 3.48 3.48 3.48 3.48 3.48 3.48 3.48 3.48 3.48 3.48 3.48 3.48
 3.48 3.48 3.48 3.48]


## Flood Simulation - BigU Lower East Side

This cell takes in storm parameters and surface volumes as well as parameters for a sea wall called the BigU and calculates the flood heights given with the effects of the BigU. The resulting output are csv files for warm and cold storm flood heights.

In [ ]:
# --- Walls + storm run (with redistribution), division-agnostic ---

import time, csv, os
start_time = time.time()

# 1) Apply coastal protection (walls) to elevation
#    Expect a CSV with at least a column of cell IDs (e.g., "ID") that correspond to your FID indexing.
walls_cfg = cfg.get("walls", {})
ParWall_ids = []

if walls_cfg.get("ids_csv"):
    wall_ids_df = pd.read_csv(walls_cfg["ids_csv"])
    # try to find an ID-like column
    id_col = next((c for c in wall_ids_df.columns if c.lower() in {"id", "fid", "cellid", "index"}), None)
    assert id_col, f"Couldn't find an ID column in {walls_cfg['ids_csv']}"
    ParWall_ids = wall_ids_df[id_col].astype(int).to_numpy().tolist()

# height can be provided in ft (legacy) or meters
ft_to_m = 0.3048
wall_h_m = float(walls_cfg.get("wall_height_m",
                    float(walls_cfg.get("wall_height_ft", 15.0)) * ft_to_m))

# Important: we don't overwrite original 'elev' so other runs can reuse it
elev_walls = elev.copy()
if ParWall_ids:
    # Index by integer FID (as in the legacy BigU_les use)
    elev_walls[np.array(ParWall_ids, dtype=int)] = elev_walls[np.array(ParWall_ids, dtype=int)] + wall_h_m

# 2) Storm surge (same mechanism as the previous block)
storm_cfg = cfg.get("storm", {})
mode = storm_cfg.get("mode", "time_history")

if mode == "time_history":
    surge_csv   = storm_cfg["surge_csv"]
    surge_col   = storm_cfg.get("surge_column", "Verified (m)")
    surge       = pd.read_csv(surge_csv)[surge_col].to_numpy().astype(float)
    nt          = int(storm_cfg.get("nt", len(surge)//2))
    assert 2*nt <= len(surge), "Surge series shorter than expected (need 2*nt points)."

    time1_w = np.arange(0, nt, dtype=float)
    time2_w = np.arange(nt, 2*nt, dtype=float)
    cpi1_w  = surge[:nt]
    cpi2_w  = surge[nt:2*nt]
    peak_w  = float(np.max(surge))

elif mode == "peak_only":
    peak_w   = float(storm_cfg["peak_m"])
    nt       = int(storm_cfg.get("nt", 24))
    t        = np.linspace(0, np.pi, 2*nt)
    surge    = peak_w * np.sin(t)
    time1_w  = np.arange(0, nt, dtype=float)
    time2_w  = np.arange(nt, 2*nt, dtype=float)
    cpi1_w   = surge[:nt]
    cpi2_w   = surge[nt:]
else:
    raise ValueError(f"Unknown storm mode: {mode}")

slr = float(storm_cfg.get("slr_m", 0.0))
if slr:
    cpi1_w = cpi1_w + slr
    cpi2_w = cpi2_w + slr
    peak_w = peak_w + slr

# 3) First pass: per-division heights/volumes with walls
fld_h_w = np.zeros(num_divs, dtype=float)
V_w     = np.zeros(num_divs, dtype=float)

ParWall_set = set(ParWall_ids)  # for fast membership checks inside FloodHeightWall

for j in range(num_divs):
    mask_j = (div_id == j)
    elev_j = elev_walls[mask_j]
    fid_j  = fid[mask_j]

    SVj = SVpieces_by_div[j]
    surfaceV_dummy = SV_all[j] if 'SV_all' in locals() and len(SV_all) > j else np.array([])

    fld_h_w[j], V_w[j] = FloodHeightWall(
        surfaceV_dummy,
        ParWall_set,  # <-- replaces BigU_les
        slope[j] if np.ndim(slope) else float(slope),
        roughness[j] if np.ndim(roughness) else float(roughness),
        SVj,
        time1_w, time2_w, cpi1_w, cpi2_w, nt,
        elev_j, fid_j,
        l, peak_w, j
    )

# 4) Redistribution using groups (prefer grouped SV curves if available)
SV_for_redistribution = [
    (SVpieces_grouped_by_div[j] if SVpieces_grouped_by_div and SVpieces_grouped_by_div[j] is not None
     else SVpieces_by_div[j])
    for j in range(num_divs)
]

fld_h_w_sect_g, V_w_sect_avr = FloodTravelSectGroup(
    peak_w,
    groups,           # dict: j -> list of division indices
    V_w,
    SV_for_redistribution
)

# 5) Save results
out_csv = cfg.get("outputs", {}).get("single_storm_with_walls_csv",
                                     "Sample_Output_Data/candidate/single_storm_flood_heights_with_walls.csv")
os.makedirs(os.path.dirname(out_csv), exist_ok=True)
with open(out_csv, "w", newline="") as out:
    writer = csv.writer(out)
    writer.writerow(fld_h_w_sect_g)

elapsed_time = time.time() - start_time
print("Elapsed Time:", elapsed_time, "sec")
print("Complete!")
print(fld_h_w_sect_g)

Elapsed Time:  0.13854122161865234 sec
Complete!
[3.48       2.14284193 3.48       2.89916265 0.94438363 2.80511712
 3.48       3.48       3.48       3.48       3.41524269 2.93767269
 2.61718049 0.         3.47241411 2.55710812 0.57955565 2.30565539]


/tmp/ipykernel_78501/2245040635.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  elev[BigU_les] = elev[BigU_les] + wallh_bigU
